In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('Preprocessed_Data.csv')

In [3]:
df

,Sentence,Sentiment,text_length,cleaned_text,tokenized_text,word_lengths,document_length,bigrams,trigrams,avg_word_length
0,The GeoSolutions technology will leverage Bene...,positive,218,geosolutions technology leverage benefon gps s...,"['geosolutions', 'technology', 'leverage', 'be...","[12, 10, 8, 7, 3, 8, 9, 8, 5, 6, 10, 9, 8, 8, ...",21,"[('geosolutions', 'technology'), ('technology'...","[('geosolutions', 'technology', 'leverage'), (...",7.714286
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,55,esi low 1 50 2 50 bk real possibility,"['esi', 'low', '1', '50', '2', '50', 'bk', 're...","[3, 3, 1, 2, 1, 2, 2, 4, 11]",9,"[('esi', 'low'), ('low', '1'), ('1', '50'), ('...","[('esi', 'low', '1'), ('low', '1', '50'), ('1'...",3.222222
2,"For the last quarter of 2010 , Componenta 's n...",positive,193,last quarter 2010 componenta net sale doubled ...,"['last', 'quarter', '2010', 'componenta', 'net...","[4, 7, 4, 10, 3, 4, 7, 7, 6, 6, 4, 7, 5, 4, 3,...",21,"[('last', 'quarter'), ('quarter', '2010'), ('2...","[('last', 'quarter', '2010'), ('quarter', '201...",5.000000
3,According to the Finnish-Russian Chamber of Co...,neutral,128,according finnish russian chamber commerce maj...,"['according', 'finnish', 'russian', 'chamber',...","[9, 7, 7, 7, 8, 5, 12, 7, 7, 9, 6]",11,"[('according', 'finnish'), ('finnish', 'russia...","[('according', 'finnish', 'russian'), ('finnis...",7.636364
4,The Swedish buyout firm has sold its remaining...,neutral,135,swedish buyout firm sold remaining 22 4 percen...,"['swedish', 'buyout', 'firm', 'sold', 'remaini...","[7, 6, 4, 4, 9, 2, 1, 7, 5, 6, 8, 5, 6, 7, 6, 7]",16,"[('swedish', 'buyout'), ('buyout', 'firm'), ('...","[('swedish', 'buyout', 'firm'), ('buyout', 'fi...",5.625000
...,...,...,...,...,...,...,...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,107,rising cost forced packaging producer huhtamak...,"['rising', 'cost', 'forced', 'packaging', 'pro...","[6, 4, 6, 9, 8, 9, 3, 2, 3, 9, 13, 5]",12,"[('rising', 'cost'), ('cost', 'forced'), ('for...","[('rising', 'cost', 'forced'), ('cost', 'force...",6.416667
5838,Nordic Walking was first used as a summer trai...,neutral,83,nordic walking first used summer training meth...,"['nordic', 'walking', 'first', 'used', 'summer...","[6, 7, 5, 4, 6, 8, 6, 5, 7, 5]",10,"[('nordic', 'walking'), ('walking', 'first'), ...","[('nordic', 'walking', 'first'), ('walking', '...",5.900000
5839,"According shipping company Viking Line , the E...",neutral,99,according shipping company viking line eu deci...,"['according', 'shipping', 'company', 'viking',...","[9, 8, 7, 6, 4, 2, 8, 11, 9, 6]",10,"[('according', 'shipping'), ('shipping', 'comp...","[('according', 'shipping', 'company'), ('shipp...",7.000000
5840,"In the building and home improvement trade , s...",neutral,88,building home improvement trade sale decreased...,"['building', 'home', 'improvement', 'trade', '...","[8, 4, 11, 5, 4, 9, 2, 1, 3, 3, 1, 2]",12,"[('building', 'home'), ('home', 'improvement')...","[('building', 'home', 'improvement'), ('home',...",4.416667


In [4]:
df.dropna(inplace=True)

In [5]:
label_encoder = LabelEncoder()
df['Sentiment_encoded'] = label_encoder.fit_transform(df['Sentiment'])
x = df['cleaned_text']
y = df['Sentiment_encoded']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
tfidf = TfidfVectorizer(max_features=10000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

In [30]:
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'Support Vector Machine': SVC(),
    'MultinomialNB': MultinomialNB(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}

param_grids = {
    'Logistic Regression': {
        'classifier__C': [0.1, 1, 10],
    },
    'Support Vector Machine': {
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    },
    'MultinomialNB': {
        'classifier__alpha': [0.5, 1.0, 1.5]
    },
    'Random Forest': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20, 30]
    },
    'XGBoost': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [3, 6, 9],
    },
    'LightGBM': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20, 30],
    }
}

In [31]:
results_list = []

for name, classifier in classifiers.items():
    print(f"Processing {name}...")
    
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', classifier)
    ])

    param_grid = param_grids.get(name, {})

    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    y_pred = grid_search.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)  # Store as dict for easier processing

    results_list.append({
        'Classifier': name,
        'Best Parameters': best_params,
        'Best Cross-Validation Score': best_score,
        'Test Accuracy': accuracy,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    })

results_df = pd.DataFrame(results_list)

print(results_df)

Processing Logistic Regression...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Processing Support Vector Machine...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Processing MultinomialNB...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Processing Random Forest...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Processing XGBoost...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Processing LightGBM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007672 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10765
[LightGBM] [Info] Number of data points in the train set: 4672, number of used features: 543
[LightGBM] [Info] Start training from score -1.915554
[LightGBM] [Info] Start training from score -0.624097
[Light

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [35]:
results_df.sort_values(by='Test Accuracy', ascending=False)

,Classifier,Best Parameters,Best Cross-Validation Score,Test Accuracy,Precision,Recall,F1-Score
1,Support Vector Machine,"{'classifier__C': 1, 'classifier__kernel': 'li...",0.689426,0.704021,0.680250,0.704021,0.673157
0,Logistic Regression,{'classifier__C': 1},0.684290,0.699743,0.673508,0.699743,0.666109
4,XGBoost,"{'classifier__max_depth': 3, 'classifier__n_es...",0.660095,0.668092,0.645170,0.668092,0.640909
2,MultinomialNB,{'classifier__alpha': 0.5},0.667382,0.666382,0.623639,0.666382,0.612094
5,LightGBM,"{'classifier__max_depth': 10, 'classifier__n_e...",0.635269,0.636441,0.626482,0.636441,0.608417
3,Random Forest,"{'classifier__max_depth': 30, 'classifier__n_e...",0.610871,0.613345,0.617855,0.613345,0.534225


In [36]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', LogisticRegression())
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.12      0.18       172
           1       0.69      0.90      0.78       626
           2       0.78      0.63      0.69       371

    accuracy                           0.70      1169
   macro avg       0.62      0.55      0.55      1169
weighted avg       0.67      0.70      0.67      1169



In [37]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', SVC(kernel='linear'))
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.41      0.13      0.20       172
           1       0.70      0.89      0.78       626
           2       0.78      0.65      0.71       371

    accuracy                           0.70      1169
   macro avg       0.63      0.56      0.56      1169
weighted avg       0.68      0.70      0.67      1169



In [38]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', MultinomialNB())
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.02      0.05       172
           1       0.65      0.98      0.78       626
           2       0.76      0.45      0.56       371

    accuracy                           0.67      1169
   macro avg       0.74      0.48      0.46      1169
weighted avg       0.71      0.67      0.60      1169



In [39]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', xgb.XGBClassifier())
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.13      0.17       172
           1       0.66      0.85      0.75       626
           2       0.80      0.58      0.67       371

    accuracy                           0.66      1169
   macro avg       0.57      0.52      0.53      1169
weighted avg       0.65      0.66      0.64      1169



In [40]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', RandomForestClassifier(n_estimators=100))
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.14      0.09      0.11       172
           1       0.66      0.79      0.72       626
           2       0.75      0.63      0.68       371

    accuracy                           0.63      1169
   macro avg       0.51      0.50      0.50      1169
weighted avg       0.61      0.63      0.62      1169



In [41]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', lgb.LGBMClassifier())
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10765
[LightGBM] [Info] Number of data points in the train set: 4672, number of used features: 543
[LightGBM] [Info] Start training from score -1.915554
[LightGBM] [Info] Start training from score -0.624097
[LightGBM] [Info] Start training from score -1.148870
              precision    recall  f1-score   support

           0       0.24      0.17      0.20       172
           1       0.66      0.78      0.71       626
           2       0.70      0.57      0.63       371

    accuracy                           0.62      1169
   macro avg       0.53      0.51      0.51      1169
weighted avg       0.61      0.62      0.61      1169



In [43]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', xgb.XGBClassifier())
])

pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.13      0.17       172
           1       0.66      0.85      0.75       626
           2       0.80      0.58      0.67       371

    accuracy                           0.66      1169
   macro avg       0.57      0.52      0.53      1169
weighted avg       0.65      0.66      0.64      1169



# Final Results

Support Vector Machine with C=1 and linear kernel has achieved highest accuracy of 70% as well as highest precision, recall and f1 score of 0.68, 0.70, 0.67 respectively